# Notebook for creating agents programatically 

### set up parent directory 

In [8]:
import sys
import os

# Add the parent directory of the current script to the Python path
cwd = os.getcwd()
dirname = os.path.dirname(cwd)
print(cwd)
print(dirname)
sys.path.append(dirname)

print(sys.path)

/Users/idekeradmin/Dropbox/GitHub/agent_evaluation/notebooks
/Users/idekeradmin/Dropbox/GitHub/agent_evaluation
['/Users/idekeradmin/Dropbox/GitHub/agent_evaluation/notebooks', '/opt/anaconda3/lib/python311.zip', '/opt/anaconda3/lib/python3.11', '/opt/anaconda3/lib/python3.11/lib-dynload', '', '/Users/idekeradmin/.local/lib/python3.11/site-packages', '/opt/anaconda3/lib/python3.11/site-packages', '/opt/anaconda3/lib/python3.11/site-packages/aeosa', '/Users/idekeradmin/Dropbox/GitHub/agent_evaluation', '/Users/idekeradmin/Dropbox/GitHub/agent_evaluation']


### Load database 

- make sure there is a ~/ae_config/config.ini file for all the configs, and ~/ae_database/ae_database.db

In [9]:
from models.analysis_plan import AnalysisPlan
from services.analysisrunner import AnalysisRunner
from app.sqlite_database import SqliteDatabase
from app.config import load_database_config
%reload_ext autoreload
%autoreload 2

# Load the db connection details
# db_type, uri, user, password = load_database_config(path='~/ae_config/test_config.ini')
# self.db = Database(uri, db_type, user, password)

_, database_uri, _, _ = load_database_config()
db = SqliteDatabase(database_uri)

### Get the available LLMs

In [10]:
llm_specs = db.find("llm")
llm_mappings = {}
for llm_spec in llm_specs:
    llm_id = llm_spec["object_id"]
    llm_properties = llm_spec["properties"]
    llm_name = llm_properties["name"]
    llm_mappings[llm_name] = llm_id

llm_mappings

{'llama-3.1-70b-versatile 2048/0.7': 'llm_a38b5b94-14d6-4216-b166-6f1515faa3a1',
 'claude-3-haiku-20240307 2048/0.7': 'llm_23a5c740-4d9b-4839-8c0a-7fb6ccae1ff1',
 'claude-3-5-sonnet-20240620 2048/0.7': 'llm_bb90988c-fa06-4a3a-99cc-81608fa8d1d7',
 'gpt-4o-mini-2024-07-18 2048/0.7': 'llm_09620189-3486-4118-941c-7c674bc25657',
 'mixtral-8x7b-32768 2048/0.7': 'llm_6e44f8e2-5089-40b1-ad37-db18f31f2b39',
 'gpt-4o-2024-05-13 2048/0.7': 'llm_dc39023b-77bf-4d2f-a6a0-829ce9c3655d',
 'llama-3.1-8b-instant 2048/0.7': 'llm_d8c65a1f-a3d9-4198-9a68-7dfb9e0b4510',
 'gemini-1.5-flash-latest': 'llm_5ca276cc-d40a-467f-8bd6-188307d3da62'}

In [15]:
cwd = os.getcwd()
dirname = os.path.dirname(cwd)

prompt_directory = (os.path.join(dirname, "prompts"))

def read_text_files(directory):
    file_contents = {}
    
    # Walk through the directory and its subdirectories
    for root, dirs, files in os.walk(directory):
        for filename in files:
            # Check if the file is a text file
            if filename.endswith('.txt'):
                # Construct the full file path
                file_path = os.path.join(root, filename)
                
                # Get the filename without extension
                file_key = os.path.splitext(filename)[0]
                
                # Read the contents of the file
                try:
                    with open(file_path, 'r', encoding='utf-8') as file:
                        content = file.read()
                        file_contents[file_key] = content
                except IOError as e:
                    print(f"Error reading file {file_path}: {e}")
    
    return file_contents

prompts = read_text_files(prompt_directory)

for name, text in prompts.items():
    print(name)

analyst_context_gradStudent_exam_0815
analyst_two_hypotheses
analyst_context_assistant_with_biocontext_0815
analyst_two_hypotheses_KG
analyst_context_assistant_0815
analyst_context_instructor_answer_0815
analyst_context_GSR_newproject_0815
context_gsr_1st_yr_ml_0814
context_gsr_0814
context_pi_0814
context_cell_editor
reviewer_reflect_0814
reviewer_0814
reviewer_cot_multi_criteria
reviewer_cot_reflect_0814
reviewer_cot_0814
reviewer_cot_reflect_novelty_0814
gs_cot
analyst_kg_brief_0821
analyst_cot_kg_brief_0821
analyst_cot_review_brief_0821
analyst_brief_0821
analyst_minimal_0814


### Create Analyst Agents

In [16]:
from models.agent import Agent

llm_claude_3_5 = llm_mappings["claude-3-5-sonnet-20240620 2048/0.7"]
llm_gpt_4o_mini = llm_mappings["gpt-4o-mini-2024-07-18 2048/0.7"]

analyst_specs = {
    "gsr_cot_review_c_3_5":{
        "llm_id": llm_claude_3_5,
        "prompt_template": prompts["analyst_cot_review_brief_0821"],
        "context": prompts["context_gsr_0814"],
        "description": ""
    },
    "gsr_cot_review_4o_mini":{
        "llm_id": llm_gpt_4o_mini,
        "prompt_template": prompts["analyst_cot_review_brief_0821"],
        "context": prompts["context_gsr_0814"],
        "description": ""
    },
    "gsr_cot_kg_3_5":{
        "llm_id": llm_claude_3_5,
        "prompt_template": prompts["analyst_cot_kg_brief_0821"],
        "context": prompts["context_gsr_0814"],
        "description": ""
    },
    "gsr_cot_kg_4o_mini":{
        "llm_id": llm_gpt_4o_mini,
        "prompt_template": prompts["analyst_cot_kg_brief_0821"],
        "context": prompts["context_gsr_0814"],
        "description": ""
    },
    "gsr_kg_3_5":{
        "llm_id": llm_claude_3_5,
        "prompt_template": prompts["analyst_kg_brief_0821"],
        "context": prompts["context_gsr_0814"],
        "description": ""
    }
}

analysts = {}

for name, spec in analyst_specs.items():
    analysts[name] = Agent.create(db, 
                                  spec["llm_id"], 
                                  spec["context"],
                                  spec["prompt_template"],
                                  name=name,
                                  description=spec.get('description'))
    
analysts

{'gsr_cot_review_c_3_5': <models.agent.Agent at 0x17d5340d0>,
 'gsr_cot_review_4o_mini': <models.agent.Agent at 0x17d51c250>,
 'gsr_cot_kg_3_5': <models.agent.Agent at 0x17d51e150>,
 'gsr_cot_kg_4o_mini': <models.agent.Agent at 0x17c3b0e10>,
 'gsr_kg_3_5': <models.agent.Agent at 0x17c3b15d0>}

### Create the Reviewer Agents

In [7]:

from models.agent import Agent

pi_context = prompts["context_pi_0814"]
reviewer_specs = {
    "rev_cot_reflect_novelty_3_5":{
        "llm_id": llm_claude_3_5,
        "prompt_template": prompts["reviewer_cot_reflect_novelty_0814"],
        "context": pi_context,
        "description": "PI reviewer preferring novelty, chain of thought, and reflection"
    },
    "rev_cot_reflect_3_5":{
        "llm_id": llm_claude_3_5,
        "prompt_template": prompts["reviewer_cot_reflect_0814"],
        "context": pi_context,
        "description": "PI reviewer using chain of thought and reflection"
    },
    "rev_cot_reflect_novelty_4o_mini":{
        "llm_id": llm_gpt_4o_mini,
        "prompt_template": prompts["reviewer_cot_reflect_novelty_0814"],
        "context": pi_context,
        "description": "PI reviewer preferring novelty, chain of thought, and reflection"
    },
    "rev_cot_reflect_4o_mini":{
        "llm_id": llm_gpt_4o_mini,
        "prompt_template": prompts["reviewer_cot_reflect_0814"],
        "context": pi_context,
        "description": "PI reviewer using chain of thought and reflection"
    },
    "rev_cot_3_5":{
        "llm_id": llm_claude_3_5,
        "prompt_template": prompts["reviewer_cot_0814"],
        "context": pi_context,
        "description": "PI reviewer, chain of thought"
    },
    "rev_cot_4o_mini":{
        "llm_id": llm_gpt_4o_mini,
        "prompt_template": prompts["reviewer_cot_0814"],
        "context": pi_context,
        "description": "PI reviewer, chain of thought"
    }
}

reviewers = {}

for name, spec in reviewer_specs.items():
    reviewers[name] = Agent.create(db, 
                                  spec["llm_id"], 
                                  spec["context"],
                                  spec["prompt_template"],
                                  name=name,
                                  description=spec.get('description'))
    
reviewers

{'rev_cot_reflect_novelty_3_5': <models.agent.Agent at 0x17d51c410>,
 'rev_cot_reflect_3_5': <models.agent.Agent at 0x17d51c790>,
 'rev_cot_reflect_novelty_4o_mini': <models.agent.Agent at 0x17d51c950>,
 'rev_cot_reflect_4o_mini': <models.agent.Agent at 0x17d51cb10>,
 'rev_cot_3_5': <models.agent.Agent at 0x17d51cd10>,
 'rev_cot_4o_mini': <models.agent.Agent at 0x17d51cf10>}

### Create a new analysis plan

### Create new analysis run 

In [18]:
'''

analysis_run = analysis_plan.generate_analysis_run(biological_context="Chromatin remodelling and transcriptional regulation (or Transcriptional regulation and chromatin remodelling)")
print(analysis_run.object_id)
print(vars(analysis_run))

runner = AnalysisRunner(db, analysis_run.object_id)
result = runner.run()
print(result)
'''

'\n\nanalysis_run = analysis_plan.generate_analysis_run(biological_context="Chromatin remodelling and transcriptional regulation (or Transcriptional regulation and chromatin remodelling)")\nprint(analysis_run.object_id)\nprint(vars(analysis_run))\n\nrunner = AnalysisRunner(db, analysis_run.object_id)\nresult = runner.run()\nprint(result)\n'

### Print the hypotheses

In [19]:
'''
hypotheses = db.load(analysis_run.object_id)[0]['hypothesis_ids']
hypotheses
'''

"\nhypotheses = db.load(analysis_run.object_id)[0]['hypothesis_ids']\nhypotheses\n"

In [20]:
'''
from models.hypothesis import Hypothesis
%reload_ext autoreload
%autoreload 2

for hypothesis_id in hypotheses:
    hypothesis = Hypothesis.load(db, hypothesis_id)
    # print(hypothesis_id)
    print(hypothesis.object_id)
    print(hypothesis.hypothesis_text)
    print(hypothesis.full_prompt)

'''

'\nfrom models.hypothesis import Hypothesis\n%reload_ext autoreload\n%autoreload 2\n\nfor hypothesis_id in hypotheses:\n    hypothesis = Hypothesis.load(db, hypothesis_id)\n    # print(hypothesis_id)\n    print(hypothesis.object_id)\n    print(hypothesis.hypothesis_text)\n    print(hypothesis.full_prompt)\n\n'